## 目录
<ol>
	<li><a href="#YouTube推荐算法">YouTube推荐算法</a></li>
    <li><a href="#FM推荐算法">FM推荐算法</a></li>
    <li><a href="#其它">其它</a></li>
    
</ol>



## YouTube推荐算法
> 原文：https://www.zhihu.com/question/20829671

### 第一阶段: User-Video 图游历算法 2008 年(这个可以划归为 用户画像 类的推荐算法)

- 数据情况是：标签少，视频多

- 构建 用户共同观看记录 （Video Co-View Graph) 图结构
	- `视频 -- 视频` 构成的图
		- 边是 同时观看两个视频的 用户数
		- 也可是 同一Session 中 被同时观看的次数
		- 也可以考虑观看顺序
		- 一些衡量视频质量的属性
            - 观看时长
            - 观看完成率（度量有多少视频被从头到位看完）
            - 丢弃率
			- 播放量
            - 转发, 点赞, 收藏等 。。。 统计信息
	- `视频 -- 用户` 构成的图


### 第二阶段: Video-Video图游历算法 2010 年

- 目标: 讲用户观看过的相似视频推荐给用户

- 数据情况: 用户行为数据
	- 用户共同观看的视频
	- 同一Session中经常被同时观看的视频
	- 第二步的基础上 考虑顺序 

- 相似度计算
    - $$r\left(v_i, v_j\right) = \frac{c_{ij}}{f\left(v_i, v_j\right)}$$
    > 其中 $c_ij$ 为所有被共同观看的次数，而 $f(v_i,v_j)$ 是一个规整化函数，试图消歧视频的流行度，因为 $v_i$ 跟 $v_j$ 中一旦存在比较Popular的结果 $c_{ij}$ 往往会偏大，一种简单的方案是将两个视频被观看的次数相乘
    - $$C_n(S) = \bigcup_{v_i \in C_{n-1}}\,\,R_i$$
    > 初步的推荐备选结果即是用户消费过视频的相似视频。如上公式，$S$ 是用户消费的视频集合，$v_i$ 为 $S$ 中的某一个视频，$R_i$ 则是 $v_i$ 对应的相似视频集。最终的备选集合 $C$，则是所有 $R_i$的并集。一般而言这种方式生成的结果作为备选的量充足的，但往往内容聚焦难以为用户找到新视频，也有备选结果不充足的情况。在这种情况下，相似视频集合可以继续扩展，从“相似视频”扩展到“相似视频的相似视频”，以此类推迭代一定的次数，得到最终的备选集合

### 第三阶段: 基于搜索及协同过滤

- 老的 相似或相关视频的计算方式
	- 协同过滤 
		- 适用于有一定的用户观看记录
		- 新视频和长尾视频 不友好

- 数据情况: 前面几阶段的数据， 视频<主题， 权重> 集合进行模式
	- 主题名来源
		- 视频描述信息
		- 上传者定义的关键词
		- 被搜索点击观看后的检索词
		- 用户定义的播放单的名字

- 新的 相似或相关视频的计算方式
	- 优点: 
        1. 用户行为依赖更小
		2. 适用于新数据及长尾数据
		3. 极大克制马太效应
	- 问题变为计算两个集合的相似度
        - $$sc\left(V_w,\,V_r\right) = q\left(V_r\right)\,\,\sum_{t \, \in \, V_w \, \cap \, V_r} T_s\left(t\right)\frac{c(t,\,V_w)}{log(1+df(t))} \, c(t,\,V_r) $$
        > 其中，$c(t,\,V)$ 表示视频 $V$ 与包含主题t的视频集合被“共同观看”的次数，也就意味着 $t$ 与 $V$ 之间的接近程度。而 $df(t)$ 则是 $t$ 出现的文档频率，$log(1+df(t))$ 用来对流行度太高的主题进行惩罚，与著名的IDF类似。$T_s\left(t\right)$是一个阀门，如果t出现在文档中的次数超过阀值，则本值为0，也就是不考虑此t的影响，反之为1，将其纳入考虑。$q(V_r)$ 则代表视频VR的质量，通过上传时间，上传者，点赞与差评的数量进行构建
        
- 第二种方法，主要思想是试图借助用户行为优化主题词的权重
    - 主题权重训练算法借鉴PairWise排序方法
        - 被推荐的视频分为两类
            - 被用户点击与观看
            - 未被点击与观看
        - 在排序时，将后者放在了前者的前面，则生成了一个错误的排序对，对应着一条训练数据X
            - 而X的长度为主题个数
            - 某一个主题出现在这个Pair对应的两条视频上，则对应的值为0
            - 如果只出现在被消费过的视频上，则为1
            - 如果只出现在没有被消费的视频上，则为-1
        - 被优化的参数W，则是X的对应系数，即主题的权重，优化的目标是在训练集合上错误最小，并加入正则化：
            - $$ \boldsymbol{L\left(w, \, S\right)} = \sum_{x\,\in \, S}log\left(1 + exp\left(-w\cdot x\right)\right) + \lambda \mid \mid w \mid \mid_1, \, $$
            - $$ w^* = arg\,\mathop{min}\limits_{w}  \boldsymbol{L\left(w, \, S_p\right)} $$
            
            
### 第四阶段：基于深度神经网络 2016年
- YouTube 推荐系统的三大难点
    1. 规模太大，简单的推荐算法在如此大规模数据量上可能是失效的；
    2. 二是实效性，即新数据不断产生，需要将其良好的呈现给用户，以平衡旧有的好内容以及新内容；
    3. 三是噪音问题，用户行为与视频描述均有噪音，并且只能获得充满噪音的用户隐含反馈，而不能直接获取用户满意度
    
- 问题建模
    - 将推荐系统定义为一个多分类器
        - 职责是确定某个用户，在某个场景与时间下，将从系统的视频中选择消费哪一个视频
        - 具体的方法是，将用户与视频全部转化为Embedding描述，即一个向量，最终用户消费某个视频的概率通过如下方式计算得到：
        - $$ P\left(w_t = i\mid U, \, C\right)\,=\,\frac{e^{v_iu}}{\sum{j\in V^{e^{v_ju}}}}  $$
        - 视频的Embedding
        - 构建用户的Embedding
            - 训练 --> 以SoftMax分类为最终优化对象
            - 将用户观看视频的Embedding整合值，搜索记录，其它信息如年龄性别等作为特征。中间为数层ReLU。
            - 利用除了用户行为外的其它信息，也是神经网络相对于普通MF类算法的优势
    - 用户观看记录作为训练数据的初始来源
        - 完成观看视频记录为正样本
            - 观看记录相对于用户的显性行为例如点赞收藏要多得多
            - 对推荐系统保留一些信息输入，以防止过渡拟合“代理问题”（即推荐系统所优化的具体指标，如点击率）
                > 例如用户往往会顺着一个检索结果页或者用户发布者浏览页进行顺序观看，然而将检索结果页面或者用户发布视频界面直接作为推荐结果呈现给用户是非常不好的
            - 所以YouTube做了一些处理规避这个问题，例如选择放弃检索句的序列信息，并其打散成词袋
                - YouTube发现，用户进行视频阅读往往是有序的
                    - 用户进行信息发现的过程往往也由流行到自己的喜好
                    - 例如用户会按照剧集的顺序进行观看，而用户进行信息发现的过程往往也由流行到自己的喜好。
                    - 于是，去预估用户的下一个观看记录，比预估用户的观看记录中中间的某一个更好，这一点也有别于传统的协同过滤

        

- 做法总结
    - 备选生成（Candidate Generation）
    > 其目标是初选结果，从海量数据中选择出符合其个人需求偏好的百级别数据
    - 排序（Ranking）
    > 通过更加丰富的用户，视频乃至场景信息，对结果进行精细化排序，得到呈现给用户的备选
    
    - 排序模块更多的是面向“场景”的，说的简单一点，就是界面
    > 用户可能在某一地方愿意点击某一条数据，但是在别的地方则不会愿意，可能在某一时间愿意点击某一条数据，但在另一个时间不会。用户观看了一个推荐界面，但是并未在这个界面上进行操作，那么随之应该进行对应内容的降级，所以对上一个推荐界面的浏览信息也可以进入到本模型中。排序的另一个职能是将各种备选联合起来。此处，需要纳入到模型中的信息更多，例如，用户最近的一次搜索词，用户最近观看的同一个主题下的视频数量，用户上一次观看同主题视频的时间，用户所使用的语言等。其架构跟备选生成阶段类似，将所有排序模型中的信息输入后，进入多层ReLU，最终进行优化的是一个加权逻辑回归层，阳性样本的权重是其观看时间。在这一层，也可以看到其推荐“代理问题”的转化，由点击行为转为了点击与观看行为结合



## FM推荐算法

### 背景
- 由Steffen Rendle在2010年提出 
- 解决线性模型的特征之间无法进行组合的问题
- 模型主要通过特征组合来解决大规模稀疏数据的分类问题
- FM和决策树都可以做特征组合，Facebook就用GBDT学习特征的自动组合
    - 决策树可以非常方便地对特征做高阶组合
    - 决策树和二项式模型有一个共同的问题 --> 无法学习到数据中不存在的模式
- 2016年的LambdaFM和2017年的Field-aware FM则分别是推荐系统和CTR预估领域的经典做法

### 出现的原因及求解
- 在LR模型中，特征之间都是独立存在的，无法体现特征之间的联系
    - 女生喜欢化妆品，男生喜欢运动产品
    > 单纯的使用 $w1∗x1+w2∗x2+..+wi∗xiw_{1}*x_{1}+w_{2}*x_{2}+..+w_{i}*x_{i}w $ 是无法实现特征之间的关联的

- 为了实现特征之间的组合，可以在线性的基础上引入二次项
    > $$ y = w_0 + \sum_{i=1}^{n}w_i∗x_i + \sum_{i=1}^{n}\sum_{j=1}^{n} w_{ij}*x_i x_j $$
    >
    > n为一个样本的特征个数。$w_{ij}$ 为特征 $x_{i}$  和 $x_{j}$ 之间的因子，共有n(n-1)/2个
    
- 求解
    - 稀疏矩阵
        - one-hot编码 --> 数据的维度大且会存在大量的0
        - 这样的稀疏矩阵对于求解非常不利
    - 对策：对于每一个特征可以引入一个辅助向量 $v_{i}$ 使其满足 $w_i = v_i v_i^T$ 其中 $v_i = \left(v_{i1}, v_{i2}, ..., v_{ik}\right)$

    > $$ y = w_0 + \sum_{i=1}^{n}w_i∗x_i + \sum_{i=1}^{n}\sum_{j=1}^{n} < v_i v_j > *x_i x_j $$
    
    - 求解向量 $< v_i, v_j >$ 特征之间的交互向量，相当于 $(a+b+c)^{2}-a_{2}-b_{2}-c_{2}$ 这样只剩下ab、ac、bc的关系了。具体过程如下
$\begin{align}
    \sum_{i=1}^{n}\sum_{j=i+1}^{n} < v_i v_j > *x_i x_j \\ 
    &= \frac{1}{2} \sum_{i=1}^{n}\sum_{j=1}^{n} < v_i v_j > *x_i x_j - \sum_{i=1}^{n} < v_i, v_i > * x_i x_i \\ 
    &= \frac{1}{2}\left( \sum_{i=1}^{n}\sum_{j=1}^{n}\sum_{f=1}^{k} v_i,fv_j,fx_i x_j - \sum_{i=1}^{n}\sum_{f=1}^{k} v_i,fv_i,fx_i x_i \right) \\
    &= \frac{1}{2} \sum_{f=1}^{k} \left( \left(\sum_{i=1}^{n} v_i, fx_i\right) \left( \sum_{j=1}^{n} v_j, fx_j \right) - \sum_{i=1}^{n} v_i^2, fx_i^2  \right) \\ 
    &= \frac{1}{2} \sum_{f=1}^{k} \left( \left(\sum_{i=1}^{n} v_i, fx_i\right)^2 - \sum_{i=1}^{n} v_i^2, fx_i^2 \right) 
\end{align}$



### FM与神经网络
- 神经网络天然地难以直接处理高维稀疏的离散特征
    - 结果: 导致神经元的连接参数太多
    - 解决：低维嵌入(embedding)
        - 词的分布式表达就是一个很好的例子
    - 神经网络对稀疏离散特征做embedding后 
        - 可以做更复杂的非线性变换，比FM更大的潜力学习到更深层的非线性关系
        - 2016年，Google提出wide and deep模型用作Google Play的app推荐
            - 利用神经网络做离散特征和连续特征之间的交叉，神经网络的输出和人工组合较低维度的离散特征一起预测
            - 采用端到端的学习，联合优化DNN和LR
- FM就可以看做对高维稀疏的离散特征做 embedding
- Cross Net
    - 为了将FM推广到高阶组合，一系列的变体被研究人员提出，例如d-way FM, 高阶FM，但是应用到实际数据中的工作一直未见报道
    - 2017年，Google的研究人员从另一种思路触发，融合了残差网络的思想，设计出叉积网络Cross Net
        - 通过加层的方式方便地扩展到任意阶数
        - 通过 embedding 层将稀疏特征转换成低维向量表示
        - 将这些向量和连续值特征拼成一个大的d维向量，作为网络的输入
        


## 其它
+ 单个用户购物习惯
+ 对订单数据进行聚类(挖掘商品与商品之间的搭配关系)
+ 挖掘用户和用户之间的相似度


+ Apriori推荐算法还是有可取性的
----------------------------------


+ 协同过滤推荐
>
>协同过滤（Collaborative Filtering Recommendation），这是目前使用的非常广泛的一个算法。分为基于用户的协同过滤算法和基于物品的协同过滤算法。
>
>系统思想是：如果用户在过去有相同的偏好（比如他们浏览或买过相同的书），那么他们在未来也会有相似的偏好。比如，用户A和B购买经历非常重叠，而且最近A买了B不知道的一本>书，那么这时的基本逻辑是像B推荐这本书。由于选择可能感兴趣的书涉及到从大量的集合中过滤出最有希望的书，而且用户是隐式地与其他人相互协作，因此这种技术被称为协同过滤。
>纯粹的协同过滤过滤方法不会利用或要求任何有关物品本身的知识，这样子的优势是，系统不需要获取并维护这些数据，其次，使用这些特征推荐的物品与用户过去喜欢的物品确实很相似，这样可能更有效一些。

>输入：用户-物品评分矩阵
>输出：当前用户对物品喜欢或不喜欢程度的预测数值，n项推荐物品列表

----------------------------------


+ 基于用户的最近邻推荐
>
>算法思想：以给定的评分数据集和当前用户为输入，寻找过去与当前用户有相似偏好的其他用户（这些用户有时也称为对等用户或者最近邻），找出后，对当前用户没有购买过/见过/使用过的每个商品p，利用其最近邻进行商品p的评分，以该评分作为预测的当前用户对这些商品的评分，并将评分高的物品推荐给当前用户。

>一句话：找到跟用户有相似兴趣的其他用户，将其他用户感兴趣的物品推荐给当前用户。
>假设：1）如果这些用户们过去有相似的偏好，那么他们未来也会有相似的偏好；2）用户的偏好不会随时间的变化而变化


----------------------------------

+ 基于物品的最近邻推荐
>
>算法思想：流程类似基于用户的协同过滤，但是是计算和利用物品之间的相似度。如果商品A和商品B很相似，即购买商品A的用户群和购买商品B的用户群重合度很高（A和B被人同时喜欢），那么A和B商品就很相似，如果发现当前用户购买过A但是没有购买过B，于是可以向用户推荐B商品。

>一句话：找到相似的物品，向用户推荐他没买过的相似的物品。


----------------------------------

+ 数据稀疏和冷启动
>
>实际应用中，由于用户一般只会评价/购买少部分的物品，评分矩阵一般都非常稀疏。此时一般需要利用用户的附加信息，如用户的性别，年龄，教育，兴趣等信息。针对刚上线的推荐系统，当然此时已经不再是纯粹的协同方法了。
>
>用户-物品关系图，计算路径距离，扩展激活
>
>缺省投票：给评分次数少的物品赋以缺省值。

冷启动：给没评分过用户推荐，给没被评分/被购买过的物体做推荐。解决的办法也是使用混合的方法（引入额外的信息）
